In [2]:
import web
import time
import logging
import json
import pandas as pd
import numpy as np
import joblib

model_path = './'
lgb_r_1d = joblib.load(model_path + 'lgb_r_1d.pkl')
lgb_r_2d = joblib.load(model_path + 'lgb_r_2d.pkl')
lgb_r_3d = joblib.load(model_path + 'lgb_r_3d.pkl')
lgb_r_4d = joblib.load(model_path + 'lgb_r_4d.pkl')
lgb_r_5d = joblib.load(model_path + 'lgb_r_5d.pkl')
lgb_r_6d = joblib.load(model_path + 'lgb_r_6d.pkl')
lgb_r_7d = joblib.load(model_path + 'lgb_r_7d.pkl')
source_lgb_b = joblib.load(model_path + 'source_lgb_b.pkl')

In [14]:
def do_source_info_1(jsondata):
    '''
    :param jsondata: {columns : 'channel_id','source_id','source_run_date','source_run_date_amount','type_0_rate',
                      'type_1_rate','type_2_rate','source_30_roi_predict'}
    :return:
    '''

    source_info_1 = pd.DataFrame(data=jsondata['data'], columns=jsondata['columns'])
    source_info_1 = source_info_1[source_info_1['source_run_date_amount']>=500]
    # 实时模型预测：差计划source_type=0；观望source_type=1
    drop_features = ['channel_id', 'source_id', 'source_run_date','source_run_date_amount']
    source_info_1['source_type'] = source_lgb_b.predict(source_info_1.drop(drop_features, axis=1))
    source_info_1['source_type'] = source_info_1['source_type'].apply(lambda x: 0 if x < 0.3 else 1)
    # 数据输出
    source_info_1 = source_info_1[['channel_id', 'source_id', 'source_run_date',
                                   'source_run_date_amount', 'source_type']]

    return {"columns": source_info_1.columns.values.tolist(), "list": source_info_1.values.tolist()}


In [4]:
jsondata = { 
      "columns": ["channel_id","source_id","source_run_date","source_run_date_amount","type_0_rate","type_1_rate","type_2_rate","source_30_roi_predict"],
      "data":[[1,1,"2020-03-30",1000,0.5,0.3,0.2,0.3],
            [2,3,"2020-03-30",1000,0.4,0.3,0.3,0.3]
      ]
    }

In [7]:
source_info_1 = pd.DataFrame(data=jsondata['data'], columns=jsondata['columns'])

In [8]:
source_info_1

,channel_id,source_id,source_run_date,source_run_date_amount,type_0_rate,type_1_rate,type_2_rate,source_30_roi_predict
0,1,1,2020-03-30,1000,0.5,0.3,0.2,0.3
1,2,3,2020-03-30,1000,0.4,0.3,0.3,0.3


In [9]:
source_info_1['source_run_date_amount']

0    1000
1    1000
Name: source_run_date_amount, dtype: int64

In [6]:
print(jsondata)

{'columns': ['channel_id', 'source_id', 'source_run_date', 'source_run_date_amount', 'type_0_rate', 'type_1_rate', 'type_2_rate', 'source_30_roi_predict'], 'data': [[1, 1, '2020-03-30', 1000, 0.5, 0.3, 0.2, 0.3], [2, 3, '2020-03-30', 1000, 0.4, 0.3, 0.3, 0.3]]}


In [15]:
do_source_info_1(jsondata=jsondata)

{'columns': ['channel_id',
  'source_id',
  'source_run_date',
  'source_run_date_amount',
  'source_type'],
 'list': [[1, 1, '2020-03-30', 1000, 1], [2, 3, '2020-03-30', 1000, 1]]}